In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial'
log_name = 'test'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 13:43:28.469233+00:00,Joe,0,1,COMPLETE,2020-01-01 14:59:41.608354+00:00,2,0 days 01:16:13.139121,4573,49408,1,0,0,0,0,0,1,0
1,REPAIR,START,2020-01-01 14:59:41.608354+00:00,Jane,0,4,COMPLETE,2020-01-04 18:38:11.639016+00:00,5,3 days 03:38:30.030662,272310,53981,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 19:33:48.900814+00:00,1,1,10,COMPLETE,2020-01-01 20:34:39.406353+00:00,11,0 days 01:00:50.505539,3650,70428,1,0,0,1,0,0,0,0
4,REPAIR,START,2020-01-01 20:34:39.406353+00:00,1,1,13,COMPLETE,2020-01-02 09:05:17.659036+00:00,14,0 days 12:30:38.252683,45038,74079,1,0,1,2,0,0,0,0
6,DIAGNOSIS,START,2020-01-02 07:47:21.536525+00:00,Clark,2,19,COMPLETE,2020-01-02 08:45:47.275476+00:00,20,0 days 00:58:25.738951,3505,28041,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,QUALITY_CONTROL,START,2022-06-30 11:11:14.473595+00:00,Jane,890,8017,COMPLETE,2022-06-30 13:45:04.355487+00:00,8018,0 days 02:33:49.881892,9229,40274,1,1,1,0,0,3,0,0
2669,QUALITY_CONTROL,START,2022-06-30 12:13:46.487067+00:00,Joe,889,8008,COMPLETE,2022-06-30 19:40:59.136223+00:00,8009,0 days 07:27:12.649156,26832,44026,1,1,1,1,0,1,1,0
2673,DIAGNOSIS,START,2022-06-30 22:25:05.097535+00:00,Karsten,891,8020,COMPLETE,2022-06-30 23:15:20.606227+00:00,8021,0 days 00:50:15.508692,3015,80705,1,0,0,0,0,0,0,1
2674,REPAIR,START,2022-06-30 23:15:20.606227+00:00,Jane,891,8023,COMPLETE,2022-07-01 07:28:48.982352+00:00,8024,0 days 08:13:28.376125,29608,83720,1,0,1,0,0,1,0,1


In [4]:
N = 10000
n_processes = 10
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

100%|████████████████████████████████████████████████████████████████████████████████| 892/892 [00:26<00:00, 33.79it/s]


  1%|▌                                                                                            | 5/892 [00:17<38:17,  2.59s/it]

  1%|█▏                                                                                          | 11/892 [00:21<20:56,  1.43s/it]

  2%|█▋                                                                                          | 16/892 [00:28<19:50,  1.36s/it]

  2%|█▊                                                                                          | 18/892 [00:33<23:39,  1.62s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 887/892 [18:17<00:05,  1.11s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████▊| 890/892 [18:18<00:01,  1.23it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 891/892 [18:18<00:00,  1.37it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [18:20<00:00,  1.06it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [18:20<00:00,  1.23s/it]

In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('-2.599931555244133025903597414')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(11731.710151253325)

In [ ]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

 64%|███████████████████████████████████████████████████▍                            | 574/892 [06:32<03:37,  1.46it/s]Process ForkPoolWorker-113:
Process ForkPoolWorker-101:
Process ForkPoolWorker-114:
Process ForkPoolWorker-111:
Process ForkPoolWorker-105:
Process ForkPoolWorker-122:
Process ForkPoolWorker-139:
Process ForkPoolWorker-134:
Process ForkPoolWorker-141:
Process ForkPoolWorker-112:
Process ForkPoolWorker-118:
Process ForkPoolWorker-144:
Process ForkPoolWorker-108:
Process ForkPoolWorker-115:
Process ForkPoolWorker-106:
Process ForkPoolWorker-128:
Process ForkPoolWorker-123:
Process ForkPoolWorker-129:
Process ForkPoolWorker-143:
Process ForkPoolWorker-109:
Process ForkPoolWorker-145:
Process ForkPoolWorker-137:
Process ForkPoolWorker-140:
Process ForkPoolWorker-127:
Process ForkPoolWorker-147:
Process ForkPoolWorker-121:
Process ForkPoolWorker-116:
Process ForkPoolWorker-138:
Process ForkPoolWorker-117:
Process ForkPoolWorker-125:
Process ForkPoolWorker-120:
Process ForkP

  0%|▏                                                                                          | 2/892 [00:14<1:29:35,  6.04s/it]

  1%|▉                                                                                            | 9/892 [00:15<14:50,  1.01s/it]

  1%|█▏                                                                                          | 11/892 [00:19<18:43,  1.28s/it]

  1%|█▏                                                                                          | 12/892 [00:24<26:57,  1.84s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 886/892 [17:33<00:06,  1.11s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████▋| 889/892 [17:34<00:02,  1.16it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▊| 890/892 [17:34<00:01,  1.26it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [17:36<00:00,  1.13it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [17:36<00:00,  1.18s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R))

In [ ]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

  0%|▏                                                                                          | 2/892 [00:17<2:03:35,  8.33s/it]

  1%|▌                                                                                            | 5/892 [00:17<36:24,  2.46s/it]

  1%|▊                                                                                            | 8/892 [00:17<18:14,  1.24s/it]

  1%|█                                                                                           | 10/892 [00:18<13:28,  1.09it/s]

  1%|█▏                                                                                          | 11/892 [00:19<15:50,  1.08s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 887/892 [18:47<00:04,  1.01it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▋| 889/892 [18:49<00:03,  1.01s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████▊| 890/892 [18:50<00:01,  1.05it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [18:51<00:00,  1.11it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [18:51<00:00,  1.27s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R_S))

In [ ]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|▏                                                                                          | 2/892 [00:19<2:23:34,  9.68s/it]

  0%|▍                                                                                            | 4/892 [00:20<59:13,  4.00s/it]

  1%|█▏                                                                                          | 11/892 [00:22<16:02,  1.09s/it]

  2%|█▍                                                                                          | 14/892 [00:23<12:22,  1.18it/s]

  2%|█▉                                                                                          | 19/892 [00:39<27:27,  1.89s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████ | 883/892 [21:17<00:07,  1.19it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████████▏| 884/892 [21:25<00:14,  1.78s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 891/892 [21:27<00:00,  1.06it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [21:29<00:00,  1.01it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [21:29<00:00,  1.45s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_R))

In [ ]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

  0%|▏                                                                                          | 2/892 [00:15<1:40:37,  6.78s/it]

  1%|█▏                                                                                          | 11/892 [00:17<13:45,  1.07it/s]

  1%|█▏                                                                                          | 12/892 [00:18<14:05,  1.04it/s]

  2%|█▋                                                                                          | 16/892 [00:25<18:59,  1.30s/it]

  2%|█▊                                                                                          | 17/892 [00:30<26:05,  1.79s/it]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████▋| 889/892 [16:19<00:02,  1.08it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▊| 890/892 [16:19<00:01,  1.19it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 891/892 [16:20<00:00,  1.08it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [16:21<00:00,  1.12it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [16:21<00:00,  1.10s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S))

In [ ]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

  0%|▏                                                                                          | 2/892 [00:16<1:45:55,  7.14s/it]

  1%|▌                                                                                            | 5/892 [00:17<30:47,  2.08s/it]

  1%|█▏                                                                                          | 11/892 [00:19<14:14,  1.03it/s]

  2%|█▍                                                                                          | 14/892 [00:19<09:51,  1.49it/s]

  2%|█▋                                                                                          | 16/892 [00:31<27:22,  1.88s/it]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████▌| 888/892 [15:45<00:02,  1.73it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▊| 890/892 [15:48<00:01,  1.19it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 891/892 [15:49<00:00,  1.07it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [15:50<00:00,  1.13it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [15:50<00:00,  1.07s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

In [ ]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|▎                                                                                            | 3/892 [00:09<35:36,  2.40s/it]

  1%|▋                                                                                            | 7/892 [00:09<11:45,  1.25it/s]

  1%|█▏                                                                                          | 11/892 [00:19<25:03,  1.71s/it]

  2%|█▋                                                                                          | 16/892 [00:20<13:26,  1.09it/s]

  2%|██▏                                                                                         | 21/892 [00:28<17:48,  1.23s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████████████████▉ | 882/892 [18:53<00:12,  1.21s/it]

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 886/892 [18:56<00:06,  1.04s/it]

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 887/892 [18:57<00:05,  1.02s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████▉| 891/892 [19:01<00:00,  1.02it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [19:01<00:00,  1.28s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

In [ ]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|▏                                                                                          | 2/892 [00:17<2:05:19,  8.45s/it]

  1%|▊                                                                                            | 8/892 [00:18<22:46,  1.55s/it]

  1%|█▏                                                                                          | 11/892 [00:19<16:52,  1.15s/it]

  2%|█▍                                                                                          | 14/892 [00:20<11:19,  1.29it/s]

  2%|█▊                                                                                          | 18/892 [00:34<28:12,  1.94s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 886/892 [18:48<00:05,  1.17it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████████▍| 887/892 [18:52<00:06,  1.27s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████▊| 890/892 [18:53<00:01,  1.16it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [18:53<00:00,  1.49it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████| 892/892 [18:53<00:00,  1.27s/it]

In [ ]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

In [ ]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

In [ ]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)